In [ ]:
from llama_index.core.agent.workflow import FunctionAgent
from llama_index.llms.openai import OpenAI
from llama_index.core.workflow import Context
import requests
import os
from typing import List, Union
from dotenv import load_dotenv

# Caricamento delle variabili d'ambiente
load_dotenv()
CHIAVE_API_OPENAI = os.getenv("OPENAI_API_KEY")
URL_API_DATI = os.getenv("DATA_API")

# Funzione condivisa per ottenere e processare i nomi delle amministrazioni
def ottieni_nomi_amministrazioni() -> Union[List[str], str]:
    """Ottiene i nomi delle amministrazioni dall'API di dati.gov.it.
    
    Restituisce:
        Union[List[str], str]: Lista di nomi delle amministrazioni o messaggio di errore.
    """
    risposta = requests.get(URL_API_DATI)
    if risposta.status_code == 200:
        dati = risposta.json()
        if dati.get("success"):
            risultato = dati.get("result", {}).get("facets", {}).get("holder_name", {}).keys()
            return list(risultato) if isinstance(risultato, dict) else risultato
        return "Errore: richiesta non riuscita."
    return f"Errore HTTP: {risposta.status_code}"

# Funzione per filtrare i nomi delle amministrazioni
def filtra_nomi_amministrazioni(nomi: List[str], parola_chiave: str) -> List[str]:
    """Filtra i nomi delle amministrazioni in base a una parola chiave.
    
    Argomenti:
        nomi (List[str]): Lista dei nomi delle amministrazioni.
        parola_chiave (str): Parola chiave per il filtro.
    
    Restituisce:
        List[str]: Lista filtrata dei nomi.
    """
    return [nome for nome in nomi if parola_chiave.lower() in nome.lower()]


# Tool: filtra i nomi delle amministrazioni per parola chiave
def Tool_filtra_amministrazioni(parola_chiave: str) -> List[str]:
    """Tool per filtrare i nomi delle amministrazioni in base a una parola chiave.
    
    Argomenti:
        parola_chiave (str): Parola chiave per il filtro.
    
    Restituisce:
        List[str]: Lista filtrata.
    """
    assert isinstance(parola_chiave, str), "La parola chiave deve essere una stringa."
    if not parola_chiave:
        raise ValueError("La parola chiave non può essere vuota.")
    if not CHIAVE_API_OPENAI or not URL_API_DATI:
        raise ValueError("Le variabili d'ambiente OPENAI_API_KEY e DATA_API devono essere impostate.")
    lista_nomi = ottieni_nomi_amministrazioni()
    return filtra_nomi_amministrazioni(lista_nomi, parola_chiave)

# Tool: conta quante amministrazioni corrispondono alla parola chiave
def Tool_conta_amministrazioni(nomi: List[str], parola_chiave: str) -> int:
    """Tool per contare le amministrazioni che corrispondono alla parola chiave.
    
    Argomenti:
        nomi (List[str]): Lista dei nomi delle amministrazioni.
        parola_chiave (str): Parola chiave per il filtro.
    
    Restituisce:
        int: Numero di corrispondenze.
    """
    if not isinstance(nomi, list):
        raise ValueError("I nomi devono essere una lista.")
    if not isinstance(parola_chiave, str):
        raise ValueError("La parola chiave deve essere una stringa.")
    if not nomi:
        return 0
    return len(filtra_nomi_amministrazioni(nomi, parola_chiave))



## react agent

In [ ]:
# Setup LLM and Agent
llm = OpenAI(model="gpt-4o")
agent = FunctionAgent(
    tools=[
        Tool_filtra_amministrazioni,
        Tool_conta_amministrazioni,
    ],
    llm=llm,
    verbose=True,
    name="DatiGovAgent",
    description="Agente per interagire con l'API di dati.gov.it.",
    system_prompt="Sei un agente che risponde a domande sui dati di holder_name da dati.gov.it. Usa i tool disponibili per ottenere le informazioni necessarie. Rispondi sempre in Italiano."
    )

ctx = Context(agent)

# Run Agent
handler = agent.run("Elenca tutte le ARPA. Conta quante sono", ctx=ctx)

Running step init_run
Step init_run produced event AgentInput
Running step setup_agent
Step setup_agent produced event AgentSetup
Running step run_agent_step
Step run_agent_step produced event AgentOutput
Running step parse_agent_output
Step parse_agent_output produced no event
Running step call_tool
Running step call_tool
Step call_tool produced event ToolCallResult
Running step aggregate_tool_results
Step aggregate_tool_results produced no event
Step call_tool produced event ToolCallResult
Running step aggregate_tool_results
Step aggregate_tool_results produced event AgentInput
Running step setup_agent
Step setup_agent produced event AgentSetup
Running step run_agent_step
Step run_agent_step produced event AgentOutput
Running step parse_agent_output
Step parse_agent_output produced event StopEvent


In [3]:
from llama_index.core.agent.workflow import AgentStream

async for ev in handler.stream_events():
    if isinstance(ev, AgentStream):
        print(f"{ev.delta}", end="", flush=True)

response = await handler

Le ARPA elencate sono:

1. ARPA
2. ARPA FVG
3. ARPA Lombardia
4. ARPA Puglia
5. ARPA Veneto
6. ARPAT - Agenzia regionale per la protezione ambientale della Toscana
7. ARPAV
8. ARPAV - Servizio Osservatorio Acque Interne
9. ARPAV - Servizio Osservatorio Aria
10. Agenzia Regionale per Le Politiche Attive del Lavoro - Arpal Umbria

In totale, ci sono 20 amministrazioni che corrispondono alla parola chiave "ARPA".